# 模型推理 - 使用 QLoRA 微调后的 ChatGLM-6B

In [4]:
import torch
from transformers import AutoModel, AutoTokenizer, BitsAndBytesConfig

# 模型ID或本地路径
model_name_or_path = 'THUDM/chatglm3-6b'

In [5]:
_compute_dtype_map = {
    'fp32': torch.float32,
    'fp16': torch.float16,
    'bf16': torch.bfloat16
}

# QLoRA 量化配置
q_config = BitsAndBytesConfig(load_in_4bit=True,
                              bnb_4bit_quant_type='nf4',
                              bnb_4bit_use_double_quant=True,
                              bnb_4bit_compute_dtype=_compute_dtype_map['bf16'])

# 加载量化后模型(与微调的 revision 保持一致）
base_model = AutoModel.from_pretrained(model_name_or_path,
                                      quantization_config=q_config,
                                      device_map='auto',
                                      trust_remote_code=True,
                                      revision='b098244')

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [6]:
base_model.requires_grad_(False)
base_model.eval()

ChatGLMForConditionalGeneration(
  (transformer): ChatGLMModel(
    (embedding): Embedding(
      (word_embeddings): Embedding(65024, 4096)
    )
    (rotary_pos_emb): RotaryEmbedding()
    (encoder): GLMTransformer(
      (layers): ModuleList(
        (0-27): 28 x GLMBlock(
          (input_layernorm): RMSNorm()
          (self_attention): SelfAttention(
            (query_key_value): Linear4bit(in_features=4096, out_features=4608, bias=True)
            (core_attention): CoreAttention(
              (attention_dropout): Dropout(p=0.0, inplace=False)
            )
            (dense): Linear4bit(in_features=4096, out_features=4096, bias=False)
          )
          (post_attention_layernorm): RMSNorm()
          (mlp): MLP(
            (dense_h_to_4h): Linear4bit(in_features=4096, out_features=27392, bias=False)
            (dense_4h_to_h): Linear4bit(in_features=13696, out_features=4096, bias=False)
          )
        )
      )
      (final_layernorm): RMSNorm()
    )
    (output_la

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path,
                                          trust_remote_code=True,
                                          revision='b098244')

## 使用原始 ChatGLM3-6B 模型

In [8]:
input_text = "如何借用电脑？"

In [9]:
response, history = base_model.chat(tokenizer, query=input_text)

In [10]:
print(response)

要借用电脑,需要先了解电脑的使用权限。如果拥有电脑的访问权限,可以按照以下步骤使用电脑:

1. 开机:按下电脑主机箱上的电源按钮,等待电脑启动。

2. 登录系统:如果使用的是Windows系统,按下“Windows键+R”组合键打开“运行”对话框,在对话框中输入“net user”并按下回车键,输入用户名和密码,点击“确定”按钮。

3. 使用电脑:当登录系统后,就可以开始使用电脑了。如果需要打开某个应用程序,可以在“开始”菜单中搜索该应用程序并点击打开。

如果使用的是Mac系统,登录系统的步骤略有不同。请参考Mac系统的使用手册或向导,以获取更详细的帮助。


## 使用微调后的 ChatGLM3-6B

### 加载 QLoRA Adapter(Epoch=3, automade-dataset(fixed)) - 请根据训练时间戳修改 timestamp 

In [13]:
from peft import PeftModel, PeftConfig

epochs = 10
timestamp = "20240319_111823"
# timestamp = "20240319_103822"

peft_model_path = f"models/{model_name_or_path}-epoch{epochs}-{timestamp}"

config = PeftConfig.from_pretrained(peft_model_path)
qlora_model = PeftModel.from_pretrained(base_model, peft_model_path)
training_tag=f"ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-{timestamp}"

In [14]:
def compare_chatglm_results(query, base_model, qlora_model, training_tag):
    base_response, base_history = base_model.chat(tokenizer, query)

    inputs = tokenizer(query, return_tensors="pt").to(0)
    ft_out = qlora_model.generate(**inputs, max_new_tokens=512)
    ft_response = tokenizer.decode(ft_out[0], skip_special_tokens=True)
    
    print(f"问题：{query}\n\n原始输出：\n{base_response}\n\n\n微调后（{training_tag}）：\n{ft_response}")
    return base_response, ft_response

### 微调前后效果对比

In [18]:
base_response, ft_response = compare_chatglm_results("借用机申请流程是什么？", base_model, qlora_model, training_tag)

问题：借用机申请流程是什么？

原始输出：
{'name': '请求机：请向贵公司IT部门开放申请，并提交申请，等待响应。如：申请一台新的笔记本电脑，需提交申请、等待响应、IT审批、领机。', 'content': '申请步骤：1.登录内部员工门户（若非员工可输入：http://company.com/selfservice）2.选择【IT资源申请】3.选择【硬件设备申请】4.填写申请表格，包括：申请型号、数量、用途、申请人、部门等。5.提交申请，等待响应。'}


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-20240319_111823）：
[gMASK]sop 借用机申请流程是什么？ 申请借用机：请在办公门户首页-IT专区-自助服务中起草 《IT资产管理审批单》，计算机类型选择借用机。若需出国，请附出境出差审批单截图。S01属地：1.申请笔记本需要先提交《一般报告审批单（智慧终端）》，说明申请笔记本的理由，签核至本部门中心长和SBU总经理，签核完成后，提交《IT资产管理审批单》并将上述签核记录作为附件，提交申请；2.申请台式机直接提交提交《IT资产管理审批单》


In [16]:
base_response, ft_response = compare_chatglm_results("怎么设置远程桌面", base_model, qlora_model, training_tag)

问题：怎么设置远程桌面

原始输出：
远程桌面允许您在计算机上远程访问另一个计算机。如果您需要使用远程桌面连接另一个计算机，请按照以下步骤操作：1. 打开电脑，点击“开始”菜单，选择“控制面板”。2. 在控制面板中，找到并点击“用户账户控制”选项。3. 接下来，在“用户账户控制”页面中，点击“管理用户”。4. 在此页面中，可以查看所有连接此计算机的用户及其密码。5. 如果您有其他用户需要连接此计算机，请为他们创建一个账户。6. 接下来，在“用户账户控制”页面中，点击“登录”。7. 在此页面中，输入连接此计算机的用户的用户名和密码，然后点击“确定”。8. 最后，在弹出的窗口中，点击“允许远程连接”。9. 现在，其他用户就可以通过远程密码连接此计算机了。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-20240319_111823）：
[gMASK]sop 怎么设置远程桌面 开启计算机远程方法：Windows7:1、在桌面或开始菜单栏中找到;计算机;右键-属性2、在弹出窗口点击左侧远程设置3、远程协助一栏勾选;允许远程协助连接这台计算机;，远程桌面一栏选择;允许运行任意版本远程桌面计算机连接;，点击确定即可Windows10:1、在桌面或打开任意文件夹左侧的;快速访问;中找到;此电脑;右键-属性2、在弹出窗口点击左侧远程设置3、在远程协助一栏勾选;允许远程协助连接这台计算机;远程桌面一栏选择;允许远程连接到此电脑;同时取消勾选下方的;仅允许运行使用网络级别身份验证的远程桌面的计算机连接;点击确定即可详细图文操作手册：http://abc.com&#34;


In [19]:
base_response, ft_response = compare_chatglm_results("电脑开机报错", base_model, qlora_model, training_tag)

问题：电脑开机报错

原始输出：
电脑开机报错，可能是硬件故障，建议进行以下操作：

1. 关闭电脑，然后将电源线从主板上拔掉，再将电源线重新插入主板，如果能够正常开机，说明问题在于电源线。

2. 如果上述操作不能解决问题，将主板上的电源线全部拔掉，再将电源线重新插入主板，如果能够正常开机，说明问题在于电源线。

3. 如果还是不能正常开机，打开电脑，在主板上的找到BIOS设置键，进入BIOS设置后，找到 boot order 设置，将其设置为默认启动设备之后，保存修改，尝试重新启动电脑。

4. 仍无法解决，检查主板上的CPU风扇是否正常工作，如果风扇不转，请联系购买新机。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-20240319_111823）：
[gMASK]sop 电脑开机报错 开机自检后提示“01: BIOS Boot Order:聚变发电厂www.fes.com.cn/www.fes.com.cn/fes_content/html/eng/homepage/index.html聚变发电厂www.fes.com.cn/www.fes.com.cn/fes_content/html/eng/homepage/index.htmlfes_content_html_eng_homepage_index.html聚变发电厂www.fes.com.cn/www.fes.com.cn/fes_content/html/eng/homepage/index.html"；如果开机自检正常，提示“01: BIOS Boot Order:聚变发电厂www.fes.com.cn/www.fes.com.cn/fes_content/html/eng/homepage/index.html聚变发电厂www.fes.com.cn/www.fes.com.cn/fes_content/html/eng/homepage/index.html"；如果开机自检正常，提示“01: BIOS Boot Order:聚变发电厂www.fes.com.cn/www.fes.com.cn/fes_content/html/eng/homepage/index.html聚变发电厂www.fes.com.cn/www.fes.com.c

过拟合他抽了